# 簡單推論
在機器學習領域裡面，量化則被用來進行值域的轉換。

* 將 16 位元的浮點數 (FP16) 量化成 8 位元的整數 (INT8)，這種將高位元的資料轉換為低位元的過程。

### 1. 假設我們有以下 FP16 數列：
* 觀察這個數列，他的值域介在 `[-0.8, 1.2]` 之間。若我們要用 INT8 來表示，那我們需要將這個值域對應到 `[-128, 127]` 之間。
* 最簡單的做法就是透過**正規化 (Normalization)** 將這些數值正規化到那個值域裡面。
* 正規化是機器學習裡面相當重要的技巧，透過將輸入正規化到 `[-1.0, 1.0]` 之間來減少偏移量，使模型更容易收斂。


In [1]:
import numpy as np

original = np.array([1.2, 0.3, -0.1, -0.8])
min_val = min(original)
max_val = max(original)
diff = max_val - min_val
print("初始陣列：", original)
print("最小值：", min_val)
print("域值間距：", diff)

初始陣列： [ 1.2  0.3 -0.1 -0.8]
最小值： -0.8
域值間距： 2.0


#### 2. 若將正規化的作法套到量化上，首先將整組數列減去**最小值** `-0.8` 得到：

In [2]:
data = original - min_val
print("修正後的陣列：", data)

修正後的陣列： [2.  1.1 0.7 0. ]


#### 3. 接著計算原本值域的**間距** `(1.2 - -0.8) = 2.0` 並將新的數列除以間距：

In [3]:
data = data / diff
print("修正後的陣列：", data)

修正後的陣列： [1.   0.55 0.35 0.  ]


#### 4. 最後縮放到 INT8 的值域 `x * 255 - 128` 並四捨五入：

In [4]:
data = data * 255 - 128
data = np.round(data).astype(int)
print("縮放到 INT8 後的陣列：", data)

縮放到 INT8 後的陣列： [ 127   12  -39 -128]


#### 5. 實際進行矩庫運算時，通常會先將數列反向量化(Dequantize)回來，也就是按照量化的流程反向計算回去：
```python
(x + 128) / 255 * 2.0 + (-0.8)
```

In [5]:
data_int8 = data
data = (data + 128) / 255 * diff + min_val
print("反向量化後陣列：", data)

反向量化後陣列： [ 1.2         0.29803922 -0.10196078 -0.8       ]


* 可以看到反量化後的數列並沒有被精準的還原，數值內容是有誤差的。
    * 因此量化技術探討的不僅只是將數值壓縮至低位元，更重要的目標在於如何減少量化所產生的誤差。
* 最小值與間距通常會用原本的資料型態存放。
* 除了精準度的考量以外，也要考慮極值的問題，避免間距或最小值因為過大過小而發生溢位的問題。
* 但也因此需要花費額外的空間來存放最小值與間距。
    * 以上例而言，四個數值省下了 `8 * 4 = 32` 個位元，但是因為額外存了兩個 FP16 的資訊，所以兩者就抵銷了。
    * 雖然一般而言不會只拿著四個數值就在那邊做量化，但還是可以使用其他方法來進一步減少額外資訊。

#### 6. 例如我們可以先對數列**取絕對值**再做量化。
* 這樣的好處在於最小值必定為零，以上例而言，數列的值域就被縮減到 `[0.0, 1.2]` 之間。
* 按照正規化的概念，應該先將數列除以 `1.2` 來正規化到 `[-1.0, 1.0]` 之間，並乘上 `127` 來得到 INT8 數列：
    ```python
    (x / 1.2) * 127
    ```
* 透過一些簡單的數學原理，可以將算式整理如下：
    ```python
    (x / 1.2) * 127
        = x * (1 / 1.2) * 127  # 改寫為乘法
        = x * 127 * (1 / 1.2)  # 根據交換律
        = x * (127 / 1.2)      # 根據結合律
    ```
* 這個 `127 / 1.2` 就是所謂的**量化常數 (Quantization Constant)**，以此例而言為 `127 / 1.2 = 105.83` ，這也是這個量化方法裡面唯一一個額外資訊。

#### 7. 將此量化常數與原本的數列相乘並四捨五入，得到以下數列：

In [6]:
q_constant = 127 / 1.2
data = original * q_constant
data = np.round(data).astype(int)
print("量化後陣列：", data)

量化後陣列： [127  32 -11 -85]


#### 8. 而反向量化也只要除以量化常數即可：

In [7]:
data = data / q_constant
print("反向量化後陣列：", data)

反向量化後陣列： [ 1.2         0.3023622  -0.10393701 -0.80314961]


* 這樣的做法誤差會稍大一些，但是實做上也比較方便。
* 這種做法也稱為**絕對最大值量化(AbsMax Quantization)**，需要計算最小值與間距的方法則被稱為**最大最小值量化(MinMax Quantization)**，兩者都是概念最單純的量化方法。
* 依照這個思維，其實也能用相同的方法達成 INT4 量化，但是誤差就更大了! 所以一般 8-Bit 以下的量化並不是單純的線性量化而已。
* 補充：
    * 像絕對最大值量化這類**以零為中心點**的量化也被稱為**對稱量化(Symmetric Quantization)**，反之則為**非對稱量化(Asymmetric Quantization)**。
        * 這兩種量化各有優缺點，在不同情況下也有各自的最佳化場景。